In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import AdamW
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# -------------------------------
# PARAMETERS
# -------------------------------
img_height, img_width = 224, 224
batch_size = 32
num_classes = 2  # Binary classification
epochs = 20

# DeiT Architecture Parameters
patch_size = 16
num_patches = (img_height // patch_size) ** 2
projection_dim = 192  # DeiT-Tiny
num_heads = 3
transformer_layers = 12
mlp_head_units = [2048, 1024]
dropout_rate = 0.0
drop_path_rate = 0.1

# -------------------------------
# DEFINE DIRECTORIES
# -------------------------------
malignant_dirs = [
    r"E:\LY Project\Multi Cancer\Data\Malignant\all_early",
    r"E:\LY Project\Multi Cancer\Data\Malignant\all_pre",
    r"E:\LY Project\Multi Cancer\Data\Malignant\all_pro",
    r"E:\LY Project\Multi Cancer\Data\Malignant\breast_malignant",
    r"E:\LY Project\Multi Cancer\Data\Malignant\colon_aca",
    r"E:\LY Project\Multi Cancer\Data\Malignant\lung_aca",
    r"E:\LY Project\Multi Cancer\Data\Malignant\lung_scc",
    r"E:\LY Project\Multi Cancer\Data\Malignant\oral_scc"
]

benign_dirs = [
    r"E:\LY Project\Multi Cancer\Data\Benign\all_benign",
    r"E:\LY Project\Multi Cancer\Data\Benign\breast_benign",
    r"E:\LY Project\Multi Cancer\Data\Benign\colon_bnt",
    r"E:\LY Project\Multi Cancer\Data\Benign\lung_bnt",
    r"E:\LY Project\Multi Cancer\Data\Benign\oral_normal"
]

# -------------------------------
# LOAD DATA FROM MULTIPLE DIRECTORIES
# -------------------------------
print("Loading data from multiple directories...")

def load_images_from_directories(directories, label, img_height, img_width):
    """Load images from multiple directories and assign same label"""
    image_paths = []
    labels = []
    
    for directory in directories:
        if not os.path.exists(directory):
            print(f"⚠ Warning: Directory not found - {directory}")
            continue
        
        # Get all image files
        files = [os.path.join(directory, f) for f in os.listdir(directory)
                if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'))]
        
        folder_name = os.path.basename(directory)
        print(f"  {folder_name}: {len(files)} images")
        
        image_paths.extend(files)
        labels.extend([label] * len(files))
    
    return image_paths, labels

# Load malignant images (label = 1)
print("\nLoading MALIGNANT images:")
malignant_paths, malignant_labels = load_images_from_directories(
    malignant_dirs, label=1, img_height=img_height, img_width=img_width
)

# Load benign images (label = 0)
print("\nLoading BENIGN images:")
benign_paths, benign_labels = load_images_from_directories(
    benign_dirs, label=0, img_height=img_height, img_width=img_width
)

# Combine all data
all_image_paths = malignant_paths + benign_paths
all_labels = malignant_labels + benign_labels

print(f"\n{'='*60}")
print(f"Total images loaded: {len(all_image_paths)}")
print(f"  Malignant: {len(malignant_paths)} ({100*len(malignant_paths)/len(all_image_paths):.1f}%)")
print(f"  Benign: {len(benign_paths)} ({100*len(benign_paths)/len(all_image_paths):.1f}%)")
print(f"{'='*60}\n")

# -------------------------------
# CREATE TF.DATA.DATASET
# -------------------------------
def load_and_preprocess_image(path, label):
    """Load and preprocess a single image"""
    # Read image file
    image = tf.io.read_file(path)
    # Decode image
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize
    image = tf.image.resize(image, [img_height, img_width])
    # Normalize to [0, 1]
    image = image / 255.0
    # Convert label to categorical
    label = tf.one_hot(label, depth=num_classes)
    return image, label

# Convert to TensorFlow dataset
path_ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_labels))

# Shuffle all data
path_ds = path_ds.shuffle(buffer_size=len(all_image_paths), seed=42)

# Split into train and validation (80-20 split)
train_size = int(0.8 * len(all_image_paths))
val_size = len(all_image_paths) - train_size

train_ds = path_ds.take(train_size)
val_ds = path_ds.skip(train_size)

print(f"Training samples: {train_size}")
print(f"Validation samples: {val_size}\n")

# Map loading and preprocessing function
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# -------------------------------
# DEIT ARCHITECTURE COMPONENTS
# -------------------------------

class PatchExtractor(layers.Layer):
    """Extracts patches from images"""
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(layers.Layer):
    """Encodes patches with linear projection and positional embeddings"""
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

class ClassTokenLayer(layers.Layer):
    """Adds a learnable class token to the sequence"""
    def __init__(self, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.projection_dim = projection_dim
        
    def build(self, input_shape):
        self.class_token = self.add_weight(
            name="class_token",
            shape=(1, 1, self.projection_dim),
            initializer="random_normal",
            trainable=True,
        )
        super().build(input_shape)
    
    def call(self, encoded_patches):
        batch_size = tf.shape(encoded_patches)[0]
        class_tokens = tf.broadcast_to(
            self.class_token, [batch_size, 1, self.projection_dim]
        )
        return tf.concat([class_tokens, encoded_patches], axis=1)

def mlp(x, hidden_units, dropout_rate):
    """Multi-layer perceptron block"""
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

class DropPath(layers.Layer):
    """Stochastic Depth / Drop Path regularization"""
    def __init__(self, drop_prob=0.0, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x, training=None):
        if self.drop_prob == 0.0 or not training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (tf.shape(x)[0],) + (1,) * (len(x.shape) - 1)
        random_tensor = keep_prob + tf.random.uniform(shape, dtype=x.dtype)
        binary_tensor = tf.floor(random_tensor)
        output = tf.divide(x, keep_prob) * binary_tensor
        return output

def create_deit_classifier(
    input_shape,
    patch_size,
    num_patches,
    projection_dim,
    num_heads,
    transformer_layers,
    mlp_head_units,
    num_classes,
    dropout_rate=0.0,
    drop_path_rate=0.1,
):
    """Creates a Data-efficient Image Transformer (DeiT) model"""
    inputs = layers.Input(shape=input_shape)
    
    # Create patches
    patches = PatchExtractor(patch_size)(inputs)
    
    # Encode patches
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    # Add class token
    encoded_patches = ClassTokenLayer(projection_dim)(encoded_patches)
    
    # Transformer blocks with stochastic depth
    dpr = [x for x in np.linspace(0.0, drop_path_rate, transformer_layers)]
    
    for i in range(transformer_layers):
        # Layer normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        # Multi-head self-attention
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=projection_dim // num_heads,
            dropout=dropout_rate,
        )(x1, x1)
        
        # Drop path
        attention_output = DropPath(dpr[i])(attention_output)
        
        # Skip connection 1
        x2 = layers.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        mlp_output = mlp(
            x3,
            hidden_units=[projection_dim * 4, projection_dim],
            dropout_rate=dropout_rate,
        )
        
        # Drop path
        mlp_output = DropPath(dpr[i])(mlp_output)
        
        # Skip connection 2
        encoded_patches = layers.Add()([mlp_output, x2])
    
    # Layer normalization
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    
    # Extract class token representation
    representation = layers.Lambda(lambda x: x[:, 0])(representation)
    
    # Classification head
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    logits = layers.Dense(num_classes, activation='softmax')(features)
    
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

# -------------------------------
# BUILD DEIT MODEL
# -------------------------------
print("Building Data-efficient Image Transformer (DeiT) model...")

model = create_deit_classifier(
    input_shape=(img_height, img_width, 3),
    patch_size=patch_size,
    num_patches=num_patches,
    projection_dim=projection_dim,
    num_heads=num_heads,
    transformer_layers=transformer_layers,
    mlp_head_units=mlp_head_units,
    num_classes=num_classes,
    dropout_rate=dropout_rate,
    drop_path_rate=drop_path_rate,
)

model.summary()

# -------------------------------
# COMPILE MODEL WITH ADAMW
# -------------------------------
print("\nCompiling model with AdamW optimizer...")

optimizer = AdamW(
    learning_rate=0.0005,
    weight_decay=0.0001
)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -------------------------------
# CALLBACKS
# -------------------------------
checkpoint_filepath = 'best_deit_cancer_model.weights.h5'

callbacks = [
    keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
]

# -------------------------------
# TRAIN MODEL
# -------------------------------
print(f"\nTraining DeiT for {epochs} epochs...")
print("="*60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("Training completed.")
print(f"Best model saved at {checkpoint_filepath}")

# -------------------------------
# EVALUATION AND METRICS
# -------------------------------
print("\nLoading best model for evaluation...")
model.load_weights(checkpoint_filepath)

print("Plotting training history...")
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='s')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('deit_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# -------------------------------
# PREDICTIONS AND EVALUATION
# -------------------------------
print("\nPredicting classes on validation data...")

# Collect predictions and true labels
y_pred_probs = []


Reorganizing cancer data into binary classification structure...

Created binary structure at: E:\LY Project\Multi Cancer\Data_Binary
  - E:\LY Project\Multi Cancer\Data_Binary\Malignant
  - E:\LY Project\Multi Cancer\Data_Binary\Benign

COPYING MALIGNANT IMAGES
Processing Malignant: all_early - 5000 images
Processing Malignant: all_pre - 5000 images


KeyboardInterrupt: 